In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
from functools import partial
from jax_tqdm import scan_tqdm

from algorithms.utils import save_state_multiagent

import numpy as np

jax.config.update("jax_default_matmul_precision", "float32")
# jax.config.update("jax_debug_nans", True)

2025-03-29 15:43:36.782854: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743259416.797600  173623 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743259416.801777  173623 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from ernestogym.envs_jax.multi_agent.env import RECEnv

In [4]:
def my_env_creator(params, battery_type, env_type='normal'):
    if env_type == 'normal':
        env = RECEnv(params, battery_type)
    return env

In [5]:
from ernestogym.envs_jax.multi_agent.utils import get_world_metadata, get_world_metadata_from_template, get_world_data

In [6]:
world_metadata = get_world_metadata_from_template('4_agents_exp_avg_prev_act_passive_no_gen')
train_params, test_params = get_world_data(world_metadata, get_train=True, get_test=True)

battery_type = 'degrading_dropflow'

train_params

yaaaas2
yaaaas2


{'num_battery_agents': 4,
 'num_passive_houses': 1,
 'batteries': [{'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 288.0,
    'temp_ambient': 298.15,
    'nominal_cost': 1500.0},
   'bounds': {'voltage': {'low': 288.0, 'high': 398.4},
    'current': {'low': -180.0, 'high': 60.0},
    'power': {'low': -71712.0, 'high': 23904.0},
    'temperature': {'low': 273.15, 'high': 323.15},
    'temp_ambient': {'low': 273.15, 'high': 323.15},
    'soc': {'low': 0.2, 'high': 1.0},
    'soh': {'low': 0.6, 'high': 1.0}},
   'init': {'voltage': 310.8,
    'current': 0.0,
    'power': 0.0,
    'temperature': 293.15,
    'temp_ambient': 293.15,
    'soc': 0.5,
    'soh': 1.0}},
  {'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 2

In [7]:
from algorithms.multi_agent_ppo import make_train, train_wrapper
from flax.core.frozen_dict import freeze, unfreeze

In [8]:
num_envs = 4
total_timesteps = 8760 * num_envs * 400

env_type = 'normal'

config = {
    'NUM_RL_AGENTS': 4,
    'NUM_BATTERY_FIRST_AGENTS': 0,
    'NUM_ONLY_MARKET_AGENTS': 0,
    'NUM_RANDOM_AGENTS': 0,
    'MAX_ACTION_RANDOM_AGENTS': 4.,

    "LR_BATTERIES": 2e-5,
    "LR_BATTERIES_MIN": 1e-7,
    'FRACTION_DYNAMIC_LR_BATTERIES': 0.7,
    "LR_REC": 2e-4,
    "LR_REC_MIN": 1e-6,
    'FRACTION_DYNAMIC_LR_REC': 1.,

    "NUM_ENVS": num_envs,
    "NUM_STEPS": 8192, #2048 #10000,
    # "TOTAL_TIMESTEPS": 100000 * num_envs,
    "TOTAL_TIMESTEPS": total_timesteps,
    "UPDATE_EPOCHS": 10,
    "NUM_MINIBATCHES": 32, #64,
    "NORMALIZE_ENV": False,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.98,
    "CLIP_EPS": 0.20,
    "ENT_COEF": 0., #0.01,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "NET_ARCH": (64, 32, 16), #(16, 16, 16),
    "LSTM_NET_ARCH": (24, 12),
    "ACTIVATION": 'tanh',
    "LSTM_ACTIVATION": 'tanh',
    "LOGISTIC_FUNCTION_TO_ACTOR": False,
    "ENV_NAME": None,
    "LR_SCHEDULE": 'cosine',
    'DEBUG': False,
    "NORMALIZE_REWARD_FOR_GAE_AND_TARGETS": False,
    "NORMALIZE_TARGETS": False,
    "NORMALIZE_ADVANTAGES": True,
    "ENV_TYPE": env_type,
    'NETWORK_TYPE_BATTERIES': 'actor_critic',
    'NETWORK_TYPE_REC': 'actor_critic',
    'USE_WEIGHT_DECAY': True,
    'NORMALIZE_NN_INPUTS': True,
    'USE_REC_RULE_BASED_POLICY': True
}

rng = jax.random.PRNGKey(42)

env = my_env_creator(train_params, battery_type, env_type=env_type)
env, train_state = make_train(config, env)

env_testing = my_env_creator(test_params, battery_type, env_type=env_type)

val_rng = jax.random.PRNGKey(51)
val_num_iters = 8670*5

# out = train_jit(rng)

config = freeze(config)

norm? False
['temperature', 'soc', 'demand', 'generation', 'buying_price', 'selling_price', 'network_REC_diff', 'sin_day_of_year', 'cos_day_of_year', 'sin_seconds_of_day', 'cos_seconds_of_day']
['demands_base_battery_houses', 'demands_battery_battery_houses', 'generations_battery_houses', 'mean_demands_base', 'mean_demands_batteries', 'mean_generations', 'sin_day_of_year', 'cos_day_of_year', 'sin_seconds_of_day', 'cos_seconds_of_day', 'network_REC_plus', 'network_REC_minus', 'exponential_average_rec_actions_prev_step']
norm batt
norm rec
norm? False
['temperature', 'soc', 'demand', 'generation', 'buying_price', 'selling_price', 'network_REC_diff', 'sin_day_of_year', 'cos_day_of_year', 'sin_seconds_of_day', 'cos_seconds_of_day']
['demands_base_battery_houses', 'demands_battery_battery_houses', 'generations_battery_houses', 'mean_demands_base', 'mean_demands_batteries', 'mean_generations', 'sin_day_of_year', 'cos_day_of_year', 'sin_seconds_of_day', 'cos_seconds_of_day', 'network_REC_plus

In [9]:
config

FrozenDict({
    NUM_RL_AGENTS: 4,
    NUM_BATTERY_FIRST_AGENTS: 0,
    NUM_ONLY_MARKET_AGENTS: 0,
    NUM_RANDOM_AGENTS: 0,
    MAX_ACTION_RANDOM_AGENTS: 4.0,
    LR_BATTERIES: 2e-05,
    LR_BATTERIES_MIN: 1e-07,
    FRACTION_DYNAMIC_LR_BATTERIES: 0.7,
    LR_REC: 0.0002,
    LR_REC_MIN: 1e-06,
    FRACTION_DYNAMIC_LR_REC: 1.0,
    NUM_ENVS: 4,
    NUM_STEPS: 8192,
    TOTAL_TIMESTEPS: 14016000,
    UPDATE_EPOCHS: 10,
    NUM_MINIBATCHES: 32,
    NORMALIZE_ENV: False,
    GAMMA: 0.99,
    GAE_LAMBDA: 0.98,
    CLIP_EPS: 0.2,
    ENT_COEF: 0.0,
    VF_COEF: 0.5,
    MAX_GRAD_NORM: 0.5,
    NET_ARCH: (64, 32, 16),
    LSTM_NET_ARCH: (24, 12),
    ACTIVATION: 'tanh',
    LSTM_ACTIVATION: 'tanh',
    LOGISTIC_FUNCTION_TO_ACTOR: False,
    ENV_NAME: None,
    LR_SCHEDULE: 'cosine',
    DEBUG: False,
    NORMALIZE_REWARD_FOR_GAE_AND_TARGETS: False,
    NORMALIZE_TARGETS: False,
    NORMALIZE_ADVANTAGES: True,
    ENV_TYPE: 'normal',
    NETWORK_TYPE_BATTERIES: 'actor_critic',
    NETWORK_TY

In [10]:
env.battery_agents

['battery_agent_0', 'battery_agent_1', 'battery_agent_2', 'battery_agent_3']

In [11]:
env.valorization_incentive_coeff

8e-06

In [12]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)

In [13]:
optimizer_batteries

StackedOptimizer(
  model=StackedActorCritic(
    act_layers=[Linear(
      bias=Param(
        value=Array(shape=(4, 64), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x76b5c650a160>,
      dot_general=<function dot_general at 0x76b7e9eb7a60>,
      dtype=None,
      in_features=11,
      kernel=Param(
        value=Array(shape=(4, 11, 64), dtype=float32)
      ),
      kernel_init=<function orthogonal.<locals>.init at 0x76b5c650a200>,
      out_features=64,
      param_dtype=<class 'jax.numpy.float32'>,
      precision=None,
      use_bias=True
    ), <PjitFunction of <function tanh at 0x76b7e9b76660>>, Linear(
      bias=Param(
        value=Array(shape=(4, 32), dtype=float32)
      ),
      bias_init=<function constant.<locals>.init at 0x76b5c5be8d60>,
      dot_general=<function dot_general at 0x76b7e9eb7a60>,
      dtype=None,
      in_features=64,
      kernel=Param(
        value=Array(shape=(4, 64, 32), dtype=float32)
      ),
      kernel_init=<

In [14]:
networks_batteries

StackedActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(4, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x76b5c650a160>,
    dot_general=<function dot_general at 0x76b7e9eb7a60>,
    dtype=None,
    in_features=11,
    kernel=Param(
      value=Array(shape=(4, 11, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x76b5c650a200>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x76b7e9b76660>>, Linear(
    bias=Param(
      value=Array(shape=(4, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x76b5c5be8d60>,
    dot_general=<function dot_general at 0x76b7e9eb7a60>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(4, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x76b5c5be85e0>,
    out_features=32,
    par

In [15]:
network_rec

RECActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(64,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x76b5c59b0220>,
    dot_general=<function dot_general at 0x76b7e9eb7a60>,
    dtype=None,
    in_features=13,
    kernel=Param(
      value=Array(shape=(13, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x76b5c59b0860>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x76b7e9b76660>>, Linear(
    bias=Param(
      value=Array(shape=(32,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x76b5c59ee480>,
    dot_general=<function dot_general at 0x76b7e9eb7a60>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x76b5c59ee660>,
    out_features=32,
    param_dtype=<clas

In [16]:
def rec_policy(rec_obs):
    net_exchange = rec_obs['generations_battery_houses'] - rec_obs['demands_base_battery_houses'] - rec_obs['demands_battery_battery_houses']

    net_exchange_plus = jnp.maximum(net_exchange, 0)
    net_exchange_minus = -jnp.minimum(net_exchange, 0)

    action_plus = net_exchange_plus / (net_exchange_plus.sum(axis=-1, keepdims=True) + 1e-8)
    action_minus = net_exchange_minus / (net_exchange_minus.sum(axis=-1, keepdims=True) + 1e-8)

    actions = (rec_obs['network_REC_plus'] > rec_obs['network_REC_minus'])[..., None] * action_minus + (rec_obs['network_REC_plus'] <= rec_obs['network_REC_minus'])[..., None] * action_plus

    actions = jnp.where(actions.sum(axis=-1, keepdims=True) == 0., jnp.ones_like(actions)/actions.shape[-1], actions)

    # jax.debug.print('{x}', x=actions)

    return actions

In [17]:
import time
rng = jax.random.PRNGKey(42)
t0 = time.time()

# with jax.checking_leaks():
# out = train_wrapper(env, config, networks_batteries, optimizer_batteries, network_rec, optimizer_rec, rng, world_metadata, validate=True, freq_val=10, val_env=env_testing, val_rng=val_rng, val_num_iters=val_num_iters, path_saving='/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER/trained_agents/')

out = train_wrapper(env, config, networks_batteries, optimizer_batteries, None, None, rng, world_metadata, rec_rule_based_policy=rec_policy, validate=True, freq_val=10, val_env=env_testing, val_rng=val_rng, val_num_iters=val_num_iters, path_saving='/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER/trained_agents/')

runner_state = out['runner_state']

print(f"time: {time.time() - t0:.2f} s")

curr_iter: 0
	r_tot: -6625.4326171875
	r_trad: -4735.6552734375
	r_deg: -2957.747802734375
	r_clip: -43.129188537597656
	r_glob: 1111.099609375
	r_rec: 11240257.0
	mean soc: 0.5861136317253113
	std actions: [0.02343542 0.02741704 0.03459907 0.04014291]
	self consumption: 11240257.0



Running for 427 iterations:   0%|          | 0/427 [00:00<?, ?it/s]

curr_iter: 10
	r_tot: -5709.89453125
	r_trad: -4280.33251953125
	r_deg: -2640.6533203125
	r_clip: 0.0
	r_glob: 1211.0919189453125
	r_rec: 12236222.0
	mean soc: 0.38653281331062317
	std actions: [0.34188214 0.29443592 0.29365703 0.2863334 ]
	self consumption: 12236222.0

curr_iter: 20
	r_tot: -5247.7685546875
	r_trad: -4019.41064453125
	r_deg: -2636.666015625
	r_clip: 0.0
	r_glob: 1408.30810546875
	r_rec: 14235546.0
	mean soc: 0.34976834058761597
	std actions: [0.6259603  0.58904666 0.5820701  0.51088285]
	self consumption: 14235546.0

curr_iter: 30
	r_tot: -5061.291015625
	r_trad: -3929.693359375
	r_deg: -2665.59423828125
	r_clip: 0.0
	r_glob: 1533.9967041015625
	r_rec: 15516528.0
	mean soc: 0.3470224142074585
	std actions: [0.7344972 0.7510721 0.7661696 0.6293786]
	self consumption: 15516528.0

curr_iter: 40
	r_tot: -4944.125
	r_trad: -3885.368896484375
	r_deg: -2673.79443359375
	r_clip: 0.0
	r_glob: 1615.0382080078125
	r_rec: 16346362.0
	mean soc: 0.33991894125938416
	std actions: [0

In [18]:
# jax.tree.map(lambda x: x.shape, out['metrics'])

In [19]:
# jax.tree.map(lambda x: jnp.isnan(x).any(), out['metrics'])

In [20]:
# jax.tree.map(lambda x: jnp.isnan(x).any(axis=jnp.arange(1, x.ndim)), out['metrics'])

In [21]:
# out['metrics']['actions']['REC_agent'][0]

In [22]:
# out['metrics']['actions']['REC_agent'][1]

In [23]:
# np.isnan(out['metrics']['actions']['REC_agent'][1]).any()

In [24]:
# jax.tree.map(lambda x: np.nonzero(np.isnan(x)), out['metrics'])

In [25]:
# out['metrics']['actions']['REC_agent']

In [26]:
# out['metrics']['pure_reward']['r_clipping'][3]

In [27]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = (runner_state.network_batteries,
                                                                       runner_state.optimizer_batteries,
                                                                       runner_state.network_rec,
                                                                       runner_state.optimizer_rec)
networks_batteries

StackedActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(4, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x76b5c650a160>,
    dot_general=<function dot_general at 0x76b7e9eb7a60>,
    dtype=None,
    in_features=11,
    kernel=Param(
      value=Array(shape=(4, 11, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x76b5c650a200>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x76b7e9b76660>>, Linear(
    bias=Param(
      value=Array(shape=(4, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x76b5c5be8d60>,
    dot_general=<function dot_general at 0x76b7e9eb7a60>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(4, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x76b5c5be85e0>,
    out_features=32,
    par